In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.utilities import SQLDatabase

from langchain.llms import Ollama
from langchain_openai import ChatOpenAI

import os

In [2]:
template = """You are a SQL expert. Based on the table schema below, write just the SQL query without the results that would answer the user's question.:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [3]:
db = SQLDatabase.from_uri("sqlite:///db/northwind-SQLite3/dist/northwind.db")

In [4]:
def get_schema(_):
    return db.get_table_info()

In [5]:
def run_query(query):
    return db.run(query)

In [6]:
api_key = os.environ.get('OPENAI_API_KEY')
model = ChatOpenAI(openai_api_key=api_key)

In [7]:
sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [8]:
sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(EmployeeID) AS TotalEmployees FROM Employees;'

In [9]:
sql_response.invoke({"question": "How many employees have been tenured for more than 11 years?"})

"SELECT COUNT(*) \nFROM Employees \nWHERE HireDate < DATE('now', '-11 years')"

In [10]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [11]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: run_query(x["query"]),
    )
    | prompt_response
    | model
)

In [12]:
result = full_chain.invoke({"question": "How many employees are there?"})
print(result)

content='There are a total of 9 employees in the database.'


In [13]:
result = full_chain.invoke({"question": "Give me the name of employees who have been tenured for more than 11 years?"})
print(result)

content='Employees who have been tenured for more than 11 years are Nancy Davolio, Andrew Fuller, Janet Leverling, and Margaret Peacock.'
